In [1]:
!pip3 install requests beautifulsoup4 lxml datetime

!pip3 install selenium
!pip3 install PyPDF2
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 61kB 2.1MB/s 
     |████████████████████████████████| 235kB 8.5MB/s 
     |████████████████████████████████| 911kB 2.8MB/s 
     |████████████████████████████████| 81kB 2.6MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=56e46c350b77805a43d813e20bda2f596815f54a295036d1050e1254e9a910e4
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 h

In [0]:
import requests
import bs4 as bs
from lxml import html

import time
import io
import PyPDF2


import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
import os

from datetime import datetime

from IPython.display import Image

In [0]:
retry_sec = 60

In [0]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

def init_selenium():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--disable-gpu");
    chrome_options.add_argument("--disable-extensions");
    chrome_options.add_argument("--window-size=1920,1080");
    chrome_options.add_argument("--kiosk")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_experimental_option('prefs',  {
    "download.default_directory": os.getcwd(),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
    }
)
    return webdriver.Chrome('chromedriver',options=chrome_options)

In [0]:
class ArticleScraper:
  wd = init_selenium()
  url = None
  page = None
  date_format = "%m/%d/%Y"
  tree = None

  def __init__(self, url, load_selenium = False, load_bs = False, load_lxml = False):
    load_all = (not load_selenium and not load_bs and not load_lxml)
    self.url = url
    req = requests.get(url, {})
    if (load_all or load_lxml):
      self.tree = html.fromstring(req.content)
    if (load_all or load_bs):
      self.page = bs.BeautifulSoup(req.text, "lxml")
    if (load_all or load_selenium):
      self.wd.get(url)

  @property
  def title(self):
    raise NotImplementedError()

  @property
  def authors(self):
    raise NotImplementedError()

  @property
  def doi(self):
    return None

  @property
  def abstract(self):
    return None

  @property
  def date(self):
    return None

  @property
  def body(self):
    return None

  @property
  def source(self):
    raise NotImplementedError()

  @property
  def source_impact_factor(self):
    return None

  @property
  def search_keyword(self):
    return None

  @property
  def categories(self):
    return None

  @property
  def licence(self):
    return None

  @property
  def acquisition_date(self):
    return datetime.now().strftime(self.date_format)

  @property
  def citations(self):
    return None

  @property
  def organization(self):
    return None

  @property
  def keywords(self):
    return None

  @property
  def references(self):
    return None

  @property
  def link(self):
    raise NotImplementedError()

  @property
  def extralinks(self):
    return None

  def parse(self, retry_sec = retry_sec):
    data = None
    while True:
      try:
        data = {'Title': self.title,
                'Authors': self.authors,
                'DOI': self.doi,
                'Abstract': self.abstract,
                'Date': self.date,
                'Full body': self.body,
                'Source': self.source,
                'Source impact factor': self.source_impact_factor,
                'Search keyword': self.search_keyword,
                'Category': self.categories,
                'Licencing': self.licence, 
                'Document acquisition date': self.acquisition_date,
                'Citations': self.citations,
                'Organization affiliated': self.organization,
                'Keywords': self.keywords,
                'References': self.references,
                'Link': self.link,
                'Extra links': self.extralinks,
        }
        break
      except Exception as e:
        self.show_error()
        print("Retrying in % d seconds..." % (retry_sec))
        time.sleep(retry_sec)
        print("Retrying now")
        continue
    return data

  def show_error(self):
    print("ERROR: while handling %s" % (self.url), e)
    self.wd.save_screenshot("temp.png")
    Image("temp.png")

